In [2]:
from pyshacl.validate import Validator
from rdflib import OWL, RDF, RDFS, SH, Graph, Namespace

In [3]:
S223 = Namespace("http://data.ashrae.org/standard223#")

In [4]:
g = Graph()
#g.parse('../223standard/models/MODEL_SP223_core-v1.0.ttl')

In [5]:
# Validation function for G36
def run_validation(g, sg):
    v = Validator(
        g,
        shacl_graph=sg,
        options={"iterate_rules": True, "advanced": True},
    )
    conforms, report_graph, report_text = v.run()


    namespace_map = {}
    for prefix, uriref in report_graph.namespaces():
        namespace_map[prefix] = Namespace(uriref)
        # Need OR for result Message
    qs = """
        SELECT ?focusNode ?resultMessage ?resultSeverity ?resultPath
        WHERE {
            BIND(s223:g36 AS ?resultSeverity) .
            ?report rdf:type sh:ValidationReport .
            ?report sh:result ?result .
            ?result sh:focusNode ?focusNode .
            OPTIONAL {?result sh:resultMessage ?resultMessage} .
            ?result sh:resultPath ?resultPath .
            ?result sh:resultSeverity ?resultSeverity .
            }
        """

    # pretty colors
    color_map = {SH.Violation: 33, SH.Info: 34, SH.Warning: 35, S223.g36: 36}

    # query
    results = sorted(report_graph.query(qs, initNs=namespace_map))
    
    prev = None
    for focusNode, resultMessage,resultSeverity, resultPath in results:
        if focusNode != prev:
            print(focusNode)
            prev = focusNode
        color = color_map[resultSeverity]
        print(f"\x1b[{color}m    {resultMessage}\x1b[0m   {resultPath}\x1b[0m")
    return report_text

# g36 VAV Cooling Only

In [6]:
g = Graph ()
g.parse('vav41.ttl', format = 'turtle')
#need to specify format depending on version of rdflib

#add model files from 223p repository if needed (input local paths and uncomment)
#g.parse('../../223standard/models/MODEL_SP223_system-v1.0.ttl')


sg = Graph()
sg.parse('g36vav41.ttl', format = 'turtle')

# Add main model files from s223 repository if needed
# sg.parse('../../223standard/models/MODEL_SP223_core-v1.0.ttl')
# sg.parse('../../223standard/models/MODEL_SP223_device-v1.0.ttl')
# sg.parse('../../223standard/models/MODEL_SP223_system-v1.0.ttl')

<Graph identifier=N620b3ed2d7554845857e33f2792c2774 (<class 'rdflib.graph.Graph'>)>

In [7]:
# Running validation, showing only when G36 rules have been violated.
report_text = run_validation(g,sg)

http://data.ashrae.org/standard223/data/g36-figure-a-1#00019
    No Discharge Flow Property, try using vav-flow property instead   http://data.ashrae.org/standard223#hasSystemConnectionPoint


In [8]:
'No Discharge Flow Property' in report_text

True

In [9]:
# Can do minor changes to rules using SPARQL queries for testing
# Here I am deactivating the discharge flow property shape, and looking for vav-flow instead
update = """ 
    DELETE DATA {
        :s223-discharge-flow sh:deactivated false .
    } """
update2 = """
    INSERT DATA {
        :s223-discharge-flow sh:deactivated true .
    }
"""

In [10]:
# Altering shacl rules using updates
sg.update(update)
sg.update(update2)

In [11]:
report_text = run_validation(g,sg)

In [12]:
'No Discharge Flow Property' in report_text

False

## Trying out Flow Or and sh:detail
This is random experimentation to get experience writing shacl shapes works. Here I'm testing out using sh:and and sh:or on a simple model file that I've created

In [14]:
file = 'sh-and-or-test.ttl'
g = Graph ()
g.parse(file)
sg = Graph()
sg.parse(file)
v = Validator(
        g,
        shacl_graph=sg,
        options={"iterate_rules": True, "advanced": True},
    )
conforms, report_graph, report_text = v.run()
print(report_text)


Validation Report
Conforms: False
Results (2):
Constraint Violation in NodeConstraintComponent (http://www.w3.org/ns/shacl#NodeConstraintComponent):
	Severity: sh:Violation
	Source Shape: test:constraint
	Focus Node: test:fail_and_succeed_or
	Value Node: test:fail_and_succeed_or
	Message: top level constraint
Constraint Violation in NodeConstraintComponent (http://www.w3.org/ns/shacl#NodeConstraintComponent):
	Severity: sh:Violation
	Source Shape: test:constraint
	Focus Node: test:fail_or_succeed_and
	Value Node: test:fail_or_succeed_and
	Message: top level constraint



## Trying out subclass of carying sh:property constraints
conclusion: targetClass is not inherited. SHACL constraints using target class can not use rdfs:subClassOf for inheritence.
However, if a shape is a class (rdfs:Class), and subshapes are also rdfs:subclasses, then inheritance works. 
Instead of using sh:targetClass for targeting, the target must be an instance (a, rdf:type) of the constraint. 
rather than
:constraint
    a sh:NodeShape;
    sh:targetClass :class .
we should have 
:constraint 
    a sh:NodeShape, rdfs:Class ;
    sh:class :class
:instance
    a :class ;
    a :constraint ;
.

This could work by ADDING the triple ":instance a :constraint" when validation is run. 
so, before validation the instance is just a :class 
:instance 
    a:class .
then before validation is run, the instance is targetted by further describing what it should be (something satisfying :constraint).
the instance is made 
:instance 
    a :class, :constraint .


In [39]:
file = 'subClass-test.ttl'
g = Graph ()
g.parse(file)
sg = Graph()
sg.parse(file)
v = Validator(
        g,
        shacl_graph=sg,
        options={"iterate_rules": True, "advanced": True},
    )
conforms, report_graph, report_text = v.run()
print(report_text)


Validation Report
Conforms: False
Results (1):
Constraint Violation in MinCountConstraintComponent (http://www.w3.org/ns/shacl#MinCountConstraintComponent):
	Severity: sh:Violation
	Source Shape: [ sh:minCount Literal("1", datatype=xsd:integer) ; sh:path test:test_prop ]
	Focus Node: test:instance
	Result Path: test:test_prop
	Message: Less than 1 values on test:instance->test:test_prop

